# Semantic Heterogeneous Database Simulations

Let's start generating random records and semantic operations, which will be used to execute performance tests

In [1]:
import time
import json
import random
import math
import pandas as pd
import numpy as np
from database_generator import DatabaseGenerator
from datetime import datetime

## Load Phase

Inserting all records generated in the semantic heterogeneous database. Please note PyMongo library may diminish performance of insertions. However, because the simulator internally uses it, it is fair to also use it on our baseline test, so these delays might net. 

Let's decide all variables for this test

In [2]:
number_of_records = 100000
number_of_versions = 5
number_of_fields = 11
number_of_values_in_domain=20

number_of_tests = 5
confidence_interval = 0.95

### First scenario
Inserting all records and adding the semantic operations afterwards. 

In [3]:
def first_scenario():    
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    start = time.time()
    d.collection.insert_many_by_dataframe(records, 'valid_from_date')

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:    
        d.collection.execute_operation(operation[0],operation[1],operation[2])
    
    end = time.time()
    d.destroy()
    return (end - start)

In [4]:
first_scenario_times = list()

for i in range(number_of_tests):
    time_taken = first_scenario()
    first_scenario_times.append(time_taken)

### Second Scenario
Loading records only after inserting semantic operations in the collection

In [5]:
def second_scenario():    
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    start = time.time()
    d.collection.insert_many_by_dataframe(records.head(10), 'valid_from_date') #initial insert

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:          
        d.collection.execute_operation(operation[0],operation[1],operation[2])

    d.collection.insert_many_by_dataframe(records.head(-10), 'valid_from_date')
    
    end = time.time()
    d.destroy()
    return (end - start)

In [6]:
second_scenario_times = list()

for i in range(number_of_tests):
    time_taken = second_scenario()
    second_scenario_times.append(time_taken)

### Baseline Scenario

In a common database scenario, the records would be just inserted as they were generated (in raw format). User would have to deal with heterogeneity afterwards, in the querying fase. Therefore, for the loading phase, only generate raw records and bulk insert into the database. 

In [7]:
def third_scenario():    
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    start = time.time()
    d.collection.insert_many_by_dataframe(records, 'valid_from_date') #initial insert    
    
    end = time.time()
    d.destroy()
    return (end - start)

In [8]:
third_scenario_times = list()

for i in range(number_of_tests):
    time_taken = third_scenario()
    third_scenario_times.append(time_taken)

## Querying tests

For the query tests, it does not matter which database (from first loading phase or from the second) is used. Both of them posess the same number of records, fields and domain values. Let's now analyse statistics in six different scenarios, just as in YCDB benchmark.

In [9]:
def generate_test_database_preinsert():
    ### Generate database just as before
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    
    d.collection.insert_many_by_dataframe(records, 'valid_from_date')

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:    
        d.collection.execute_operation(operation[0],operation[1],operation[2])  
       
    
    return d

In [10]:
def generate_test_database_postinsert():
    ### Generate database just as before
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    
    d.collection.insert_many_by_dataframe(records.head(10), 'valid_from_date') #initial insert

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:          
        d.collection.execute_operation(operation[0],operation[1],operation[2])

    d.collection.insert_many_by_dataframe(records.head(-10), 'valid_from_date')    
    
    return d    

In [30]:
def update_and_read_test(percent_of_update):
    ### Generate database just as before
    print('Generating database')
    d = generate_test_database_preinsert()   

    updates = math.floor(100*percent_of_update)
    reads = 100-updates

    sequence = ([True]*updates)
    sequence.extend([False]*reads)
    random.shuffle(sequence)    

    records = [d.generate_record() for i in range(updates)]
    queries = []

    print('Updates:' + str(updates))
    print('Reads:' + str(reads))

    for i in range(reads):
        print('Creating operations')
        field = (random.choice(d.fields))[0]
        value = random.choice(d.field_domain[field])
        queries.append({field:value})
        print(str({field:value}))

    start = time.time()
    for operation in sequence:   
        print('Running operation')     
        if operation: # insert - Nos nossos casos de uso não faz muito sentido deleções e updates. 
            record = records.pop()
            d.collection.insert_one(json.dumps(record, default=str),record['valid_from_date'])            
        else:
            d.collection.find_many(queries.pop()) ##Nao to considerando a presença ou ausência de índices

        print('ok')

    end = time.time()
    
    d.destroy()
    print (str(end-start))
    return (end - start)
    

### Scenario 1 - Writes Heavy

In the first update scenario, a workload of 50% of reads and 50% of writes.

Note there is a difference between the test using the first generation method and the second one, due to lazy insertion.

In [31]:
write_heavy_times = list()

for i in range(number_of_tests):
    time_taken = update_and_read_test(0.5)
    write_heavy_times.append(time_taken)

write_heavy_times

Generating database
Updates:50
Reads:50
Creating operations
{'rplhr': 0.4387773262700035}
Creating operations
{'rplhr': 2.5190890314344347}
Creating operations
{'lbpcp': 'qlkzikhtufzhaubvbskwuqbscfdziolict'}
Creating operations
{'yrzaw': 0.38895832083272613}
Creating operations
{'gzveo': '2078-10-09'}
Creating operations
{'chlec': 1.70866047464939}
Creating operations
{'rplhr': 1.849106978084322}
Creating operations
{'xvpvw': 'pdvrdxblanxtltaflncsobeoiicnssxcpo'}
Creating operations
{'clcem': '2032-03-06'}
Creating operations
{'pxyoc': 'dtonztjkwtkvznglrtyhnwfawclgwrlpet'}
Creating operations
{'yrzaw': 2.078627724475594}
Creating operations
{'pxyoc': 'pyfbrcdtubhjyowzuocxksyjxnahtmcqmu'}
Creating operations
{'clcem': '2006-05-11'}
Creating operations
{'rplhr': 0.48188540432584914}
Creating operations
{'yrzaw': 22.672051952472305}
Creating operations
{'yrzaw': 1.084650015182972}
Creating operations
{'lbpcp': 'dwyixupxjoooyagmnrecrhingfmpisoski'}
Creating operations
{'clcem': '2010-09-30

KeyboardInterrupt: 